# Quantum Annealing
Quantum annealing is a computational method utilized to solve complex optimization problems, leveraging quantum mechanics. This technique takes advantage of quantum superposition and entanglement to find the optimum solution from among a vast search space more efficiently than classical computing methods, through a process called adiabatic evolution.

Adiabatic evolution in quantum annealing is a critical phase that provides the mechanism for the quantum system to seek out the minimal energy state, or the optimal solution for a given problem. The term "adiabatic" refers to the gradual change of parameters in the system, slow enough that the system remains close to its ground state. This process involves initiating a quantum state in a simple landscape where the ground state is easy to populate, and slowly transforming this into a complex landscape correlating to the problem at hand. If the evolution is slow enough, according to the adiabatic theorem, the system will stay in its ground state throughout the process, ultimately arriving at the solution when the complex landscape is achieved.

Since we have no Quantum Annealing hardware (like D-Wave) available, we are performing Simulated Quantum Annealing.

## Simulated Quantum Annealer with siquan
In this example we are using the [SiQuAn library](https://github.com/PlanQK/SimulatedQuantumAnnealing) for the Simulated Quantum Annealing. Even though it is written in C++, it can be called from Python with the corresponding plugin.
### Setup
Install cmake and clone git
```
sudo apt-get -y install cmake
git clone https://github.com/PlanQK/SimulatedQuantumAnnealing.git
```

Update and compile
```
cd SimulatedQuantumAnnealing/
git submodule init && git submodule update
cmake .
make
```

Add library to path:

In [1]:
import sys
sys.path.append("<full path>/SimulatedQuantumAnnealing/pythonInterface")

### Basic siquan example

In [2]:
import siquan

dtsqa = siquan.DTSQA()
# possibility to set various parameters
dtsqa.setHSchedule("[0.0]")
dtsqa.setTSchedule("[10, 0.001]")
dtsqa.setSeed(42)
dtsqa.setSteps(10000)
dtsqa.setTrotterSlices(32)
# first param: problem description: List of Tuples
#              each Tuple describes one interaction,
#              first param: coupling strength
#              second param: List of qubit indices
# second param: number of qubits
result = dtsqa.minimize(
    [(1,[0]), (1.5, [0,1]), (-1., [1,2,3])],
    4
)

### CNF siquan example

In [8]:
import sympy
import numpy as np
import collections
import itertools


class IsingFromCNF:
    """
    Use sympy for the transformation.
    """

    def __init__(self, cnf):
        """
        Immediately create the cost function from the cnf. This is stored in
        a dictionary where the key is a set of spin indices and the value is the
        interaction strength
        """
        self.costFunction = collections.defaultdict(float)

        # find the number of variables
        variable_ids = set()
        for interactions in itertools.chain(cnf):
            for literalID in interactions:
                variable_ids.add(abs(literalID))
        self.maxEncounteredIndex = len(variable_ids)

        self._translate_cnf(cnf)

    def _translate_cnf(self, cnf):
        for constraint in cnf:
            for key, interaction in self._translate_clause(constraint).items():
                self.costFunction[key] += interaction

    def to_ising_representation(self):
        """
        Given the internal cost function reformulate the cost for the SQA optimizer.
        """
        return [(-1 * interaction, list(spins)) for spins, interaction in self.costFunction.items()]

    def constant_term(self):
        """
        Determine the cost contribution of the constant term
        """
        return self.costFunction.get(tuple(), 0)

    def total_variables(self):
        return self.maxEncounteredIndex

    def _translate_clause(self, clause):
        """
        This function converts a clause into a polynomial given the above derivation.
        """
        polynomial_contributions = [self._binary_to_spin_var(literal) for literal in clause]
        total_polynomial = np.prod(polynomial_contributions)
        monomials = total_polynomial.monoms()
        coefficients = total_polynomial.coeffs()
        symbols = [int(str(symb)[2:]) for symb in total_polynomial.gens]
        list_of_interactions = collections.defaultdict(float)
        for i in range(len(monomials)):
            key = set()
            for monomialId, value in enumerate(monomials[i]):
                if value == 1:
                    key.add(symbols[monomialId])
            list_of_interactions[tuple(sorted(list(key)))] += coefficients[i]
        return list_of_interactions

    @classmethod
    def _binary_to_spin_var(cls, literal_id):
        """
        Transform each literal with index i to an Ising spin index. We take care
        of performing a not operation if the literal is negative and transform
        between different conventions.
        - Pysat starts counting the literal indices at 1 we start counting at 0.
        - As each literal is part of a disjunction we need to transform the
        non-negated literals with the transformation 1-s.
        """
        x = sympy.symbols(f"x_{abs(literal_id) - 1}")
        if literal_id > 0:
            return sympy.poly((1 - x) / 2)
        else:
            return sympy.poly((1 + x) / 2)

Make sure 
```
<full path>/.venv/lib/python3.10/site-packages
```
is part of the path.

In [4]:
from pysat.formula import CNF

cnf = CNF(from_clauses=[[-1, -2, -3], [-1, -2, 3]])

Set simulator

In [ ]:
dtsqa = siquan.DTSQA()
dtsqa.setHSchedule("[0.0]")
dtsqa.setTSchedule("[10, 0.001]")
dtsqa.setSeed(42)
dtsqa.setSteps(10000)
dtsqa.setTrotterSlices(32)

In [9]:
sampleProblem = IsingFromCNF(cnf)
solution = dtsqa.minimize(sampleProblem.to_ising_representation(), sampleProblem.total_variables())

# the constant term is not added to the energy for the optimizer.
# The total cost is 0 which indicates that all clauses are satisfied
print(solution)
print(eval(solution["energy"]) - sampleProblem.constant_term())